# End-to-End Time Series Forecasting Workshop

## Complete Workflow: From EDA to Production-Ready Models

This notebook demonstrates a complete time series forecasting workflow, combining everything we've learned:

### What We'll Cover:
1. **Exploratory Data Analysis** - Understanding patterns, seasonality, and trends
2. **Simple Baseline Models** - Naive methods for benchmarking
3. **Statistical Models** - AutoETS for automatic component selection
4. **Cross-Validation** - Robust model evaluation with MASE and RMSSE metrics
5. **Global ML Models** - Lag selection and feature engineering
6. **Final Comparison** - Selecting the best model based on multiple metrics

### Dataset:
- **Top 50 food items** from M5 Forecasting Competition
- Aggregated across all stores
- Daily sales from 2013 to 2016

Let's begin! 🚀

## 1. Setup and Imports

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose

# StatsForecast - simple and statistical models
from statsforecast import StatsForecast
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    WindowAverage,
    SeasonalWindowAverage,
    AutoETS
)

# MLForecast - global machine learning models
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences, LocalStandardScaler

# Machine Learning Models
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb

# Plotting settings
plt.style.use('ggplot')
sns.set_palette("husl")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

print("✅ All libraries imported successfully!")

## 2. Load Data

We're using the **top 50 food items** dataset created in the data preparation notebook.

In [ ]:
# Load the data
data_path = '/home/filtheo/Cloud-for-AI/workshop_ml_2025/data/converted_df_2.csv'
df = pd.read_csv(data_path, parse_dates=['date'])

print(f"Dataset shape: {df.shape}")
print(f"Number of unique items: {df['unique_id'].nunique()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Total days: {df['date'].nunique()}")
print(f"\nFirst few rows:")
df.head(15)

In [ ]:
# Basic statistics per item
print("Basic Statistics per Item:")
print("="*70)
item_stats = df.groupby('unique_id')['y'].agg(['count', 'mean', 'std', 'min', 'max']).round(1)
item_stats.head(20)

## 3. Exploratory Data Analysis

### 3.1 Visualize Multiple Time Series

Let's look at sales patterns for a sample of items.

In [ ]:
# Select 12 items for visualization (a diverse sample)
items = df['unique_id'].unique()
sample_items = items[::len(items)//12][:12]  # Select 12 evenly spaced items

# Plot all samples in a grid
fig, axes = plt.subplots(4, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, item in enumerate(sample_items):
    item_data = df[df['unique_id'] == item].sort_values('date')
    
    axes[idx].plot(item_data['date'], item_data['y'], 
                   color='black', linewidth=0.8, alpha=0.8)
    axes[idx].set_title(f'{item}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=8)
    axes[idx].set_ylabel('Daily Sales', fontsize=8)
    axes[idx].tick_params(labelsize=7)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Sample Items: Daily Sales Patterns', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n🔍 What patterns do you notice?")
print("- Different items have different sales levels")
print("- Some show strong trends, others are more stable")
print("- Weekly patterns are visible in many series")

### 3.2 Time Series Decomposition

Let's decompose a few items to see their **trend**, **seasonality**, and **residual** components.

In [ ]:
# Decompose 3 representative items
decompose_items = sample_items[:3]

for item_name in decompose_items:
    item_data = df[df['unique_id'] == item_name].sort_values('date').set_index('date')
    
    # Perform seasonal decomposition (period=7 for weekly seasonality)
    decomposition = seasonal_decompose(item_data['y'], model='additive', period=7)
    
    # Plot decomposition
    fig, axes = plt.subplots(4, 1, figsize=(14, 10))
    
    # Observed
    axes[0].plot(decomposition.observed, color='black', linewidth=1, alpha=0.8)
    axes[0].set_ylabel('Observed', fontsize=10)
    axes[0].set_title(f'Time Series Decomposition: {item_name}', fontsize=13, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Trend
    axes[1].plot(decomposition.trend, color='steelblue', linewidth=1.5, alpha=0.8)
    axes[1].set_ylabel('Trend', fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    # Seasonal
    axes[2].plot(decomposition.seasonal, color='green', linewidth=1, alpha=0.8)
    axes[2].set_ylabel('Seasonal', fontsize=10)
    axes[2].grid(True, alpha=0.3)
    
    # Residual
    axes[3].plot(decomposition.resid, color='red', linewidth=0.8, alpha=0.7)
    axes[3].set_ylabel('Residual', fontsize=10)
    axes[3].set_xlabel('Date', fontsize=10)
    axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Component Analysis for {item_name}:")
    print(f"- Trend: Shows overall direction over time")
    print(f"- Seasonal: Weekly pattern repeating every 7 days")
    print(f"- Residual: Random noise around zero\n")

### 3.3 Seasonality Strength Analysis

**Seasonality Strength = 1 - Var(Residual) / Var(Seasonal + Residual)**

- Value close to **1** = **Strong seasonality**
- Value close to **0** = **Weak seasonality**

In [ ]:
# Calculate seasonality strength for all items
seasonality_strength = {}

for item in items:
    item_data = df[df['unique_id'] == item].sort_values('date').set_index('date')
    decomp = seasonal_decompose(item_data['y'], model='additive', period=7)
    
    # Calculate strength
    seasonal_var = np.var(decomp.seasonal.dropna())
    resid_var = np.var(decomp.resid.dropna())
    
    strength = 1 - (resid_var / (seasonal_var + resid_var))
    seasonality_strength[item] = strength

# Convert to DataFrame
strength_df = pd.DataFrame(
    list(seasonality_strength.items()), 
    columns=['Item', 'Seasonality_Strength']
).sort_values('Seasonality_Strength', ascending=False)

print("Seasonality Strength by Item (Top 15):")
print("="*50)
print(strength_df.head(15).to_string(index=False))
print(f"\nAverage strength: {strength_df['Seasonality_Strength'].mean():.3f}")
print(f"Median strength: {strength_df['Seasonality_Strength'].median():.3f}")

In [ ]:
# Plot aggregated seasonality strength distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(strength_df['Seasonality_Strength'], bins=20, 
             color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(x=strength_df['Seasonality_Strength'].mean(), 
                color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {strength_df["Seasonality_Strength"].mean():.3f}')
axes[0].set_xlabel('Seasonality Strength', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Seasonality Strength', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(strength_df['Seasonality_Strength'], vert=True)
axes[1].set_ylabel('Seasonality Strength', fontsize=11)
axes[1].set_title('Seasonality Strength: Overall Summary', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("- Most items show moderate to strong weekly seasonality")
print("- This indicates weekly patterns should be captured by our models")
print("- Seasonal models (SeasonalNaive, ETS with seasonality) should perform well")

### 3.4 Day of Week Analysis

Let's examine sales patterns by day of week.

In [ ]:
# Add day of week
df_dow = df.copy()
df_dow['day_of_week'] = pd.to_datetime(df_dow['date']).dt.day_name()
df_dow['dow_num'] = pd.to_datetime(df_dow['date']).dt.dayofweek

# Calculate average sales by day of week (aggregated across all items)
dow_avg = df_dow.groupby(['dow_num', 'day_of_week'])['y'].mean().reset_index()
dow_avg = dow_avg.sort_values('dow_num')

print("Average Sales by Day of Week (All Items Combined):")
print("="*60)
print(dow_avg[['day_of_week', 'y']].to_string(index=False))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['steelblue']*5 + ['lightcoral']*2  # Highlight weekends
ax.bar(dow_avg['day_of_week'], dow_avg['y'], 
       color=colors, alpha=0.8, edgecolor='black', linewidth=1.2)

ax.set_xlabel('Day of Week', fontsize=11)
ax.set_ylabel('Average Daily Sales', fontsize=11)
ax.set_title('Average Sales by Day of Week (All Items)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Find peak day
peak_day = dow_avg.loc[dow_avg['y'].idxmax(), 'day_of_week']
low_day = dow_avg.loc[dow_avg['y'].idxmin(), 'day_of_week']
print(f"\n📈 Highest sales: {peak_day}")
print(f"📉 Lowest sales: {low_day}")

## 4. Train/Test Split

We'll hold out the **last 14 days** for testing.

In [ ]:
# Train/Test Split
test_size = 14  # 2 weeks
max_date = df['date'].max()
split_date = max_date - pd.Timedelta(days=test_size - 1)

df_train = df[df['date'] < split_date].copy()
df_test = df[df['date'] >= split_date].copy()

print(f"Train/Test Split:")
print("="*60)
print(f"Total observations: {len(df):,}")
print(f"Training set: {len(df_train):,} observations")
print(f"Test set: {len(df_test):,} observations")
print(f"\nDate ranges:")
print(f"  Train: {df_train['date'].min()} to {df_train['date'].max()}")
print(f"  Test:  {df_test['date'].min()} to {df_test['date'].max()}")
print(f"\nForecast horizon: {test_size} days")

## 5. Define Custom Evaluation Metrics

We'll use two scaled error metrics:

### 5.1 MASE (Mean Absolute Scaled Error)

$$MASE = \frac{1}{h} \sum_{t=1}^{h} \frac{|y_t - \hat{y}_t|}{\frac{1}{T-1}\sum_{i=2}^{T}|y_i - y_{i-1}|}$$

- **Numerator**: Mean absolute error of forecasts
- **Denominator**: Mean absolute error of naive forecast on training data
- **Interpretation**: MASE < 1 means better than naive forecast

### 5.2 RMSSE (Root Mean Squared Scaled Error)

$$RMSSE = \sqrt{\frac{\frac{1}{h}\sum_{t=1}^{h}(y_t - \hat{y}_t)^2}{\frac{1}{T-1}\sum_{i=2}^{T}(y_i - y_{i-1})^2}}$$

- **Numerator**: Root mean squared error of forecasts
- **Denominator**: Root mean squared error of naive forecast on training data
- **Interpretation**: RMSSE < 1 means better than naive forecast

In [ ]:
def calculate_mase(y_true, y_pred, y_train, seasonality=1):
    """
    Calculate Mean Absolute Scaled Error (MASE).
    
    Parameters:
    -----------
    y_true : array-like
        Actual values in test period
    y_pred : array-like
        Predicted values
    y_train : array-like
        Training data used to compute scaling factor
    seasonality : int, default=1
        Seasonal period for naive forecast (1 for non-seasonal)
    
    Returns:
    --------
    float : MASE value
    """
    # Calculate forecast errors
    mae_forecast = np.mean(np.abs(y_true - y_pred))
    
    # Calculate naive forecast errors on training data
    naive_errors = np.abs(np.diff(y_train, n=seasonality))
    mae_naive = np.mean(naive_errors)
    
    # Avoid division by zero
    if mae_naive == 0:
        return np.nan
    
    return mae_forecast / mae_naive


def calculate_rmsse(y_true, y_pred, y_train, seasonality=1):
    """
    Calculate Root Mean Squared Scaled Error (RMSSE).
    
    Parameters:
    -----------
    y_true : array-like
        Actual values in test period
    y_pred : array-like
        Predicted values
    y_train : array-like
        Training data used to compute scaling factor
    seasonality : int, default=1
        Seasonal period for naive forecast (1 for non-seasonal)
    
    Returns:
    --------
    float : RMSSE value
    """
    # Calculate forecast errors
    mse_forecast = np.mean((y_true - y_pred)**2)
    
    # Calculate naive forecast errors on training data
    naive_errors = np.diff(y_train, n=seasonality)**2
    mse_naive = np.mean(naive_errors)
    
    # Avoid division by zero
    if mse_naive == 0:
        return np.nan
    
    return np.sqrt(mse_forecast / mse_naive)


print("✅ Custom metrics defined:")
print("  - MASE: Mean Absolute Scaled Error")
print("  - RMSSE: Root Mean Squared Scaled Error")
print("\n💡 Both metrics are scale-free and compare against naive forecast")
print("   Values < 1.0 indicate better performance than naive baseline")

## 6. Simple Baseline Models (StatsForecast)

Let's start with simple baseline methods:
- **Naive**: Repeat last value
- **Seasonal Naive**: Repeat same day from last week
- **Window Average**: Average of last N days
- **Seasonal Window Average**: Average of same weekday from last N weeks

In [ ]:
# Prepare data for StatsForecast (rename 'date' to 'ds')
df_train_sf = df_train.rename(columns={'date': 'ds'})
df_test_sf = df_test.rename(columns={'date': 'ds'})

# Define simple baseline models
simple_models = [
    Naive(),
    SeasonalNaive(season_length=7),
    WindowAverage(window_size=7),
    SeasonalWindowAverage(season_length=7, window_size=2)
]

# Fit all models
print("Training simple baseline models...")
sf_simple = StatsForecast(
    models=simple_models,
    freq='D',
    n_jobs=-1
)

# Generate forecasts
forecasts_simple = sf_simple.forecast(df=df_train_sf, h=test_size)

print("✅ Simple models trained and forecasted!")
print(f"\nForecast shape: {forecasts_simple.shape}")
print(f"Columns: {list(forecasts_simple.columns)}")
forecasts_simple.head(10)

## 7. Statistical Models (StatsForecast)

Now let's add **AutoETS** - automatic exponential smoothing with optimal component selection.

In [ ]:
# Define statistical model
statistical_models = [
    AutoETS(season_length=7)
]

# Fit model
print("Training AutoETS...")
sf_ets = StatsForecast(
    models=statistical_models,
    freq='D',
    n_jobs=-1
)

# Generate forecasts
forecasts_ets = sf_ets.forecast(df=df_train_sf, h=test_size)

print("✅ AutoETS trained and forecasted!")
forecasts_ets.head(10)

## 8. Cross-Validation Setup

We'll use **time series cross-validation** with 2 validation windows:

```
Training data: [----train1----][val1]
               [------train2------][val2]
```

This gives us a robust estimate of model performance.

In [ ]:
def evaluate_model_cv(forecasts, actuals, train_data, model_name):
    """
    Evaluate model using MASE and RMSSE metrics with cross-validation.
    
    Parameters:
    -----------
    forecasts : DataFrame
        Forecasts from StatsForecast or MLForecast
    actuals : DataFrame
        Actual test values
    train_data : DataFrame
        Training data for scaling factor
    model_name : str
        Name of the model column in forecasts
    
    Returns:
    --------
    dict : {'mase': float, 'rmsse': float}
    """
    # Merge forecasts with actuals
    if 'ds' in forecasts.columns:
        merged = forecasts.reset_index().merge(
            actuals[['unique_id', 'ds', 'y']], 
            on=['unique_id', 'ds']
        )
    else:
        merged = forecasts.merge(
            actuals[['unique_id', 'ds', 'y']], 
            on=['unique_id', 'ds']
        )
    
    mase_scores = []
    rmsse_scores = []
    
    # Calculate metrics for each unique_id
    for uid in merged['unique_id'].unique():
        # Get test data for this item
        item_data = merged[merged['unique_id'] == uid].sort_values('ds')
        y_true = item_data['y'].values
        y_pred = item_data[model_name].values
        
        # Get training data for this item
        train_item = train_data[train_data['unique_id'] == uid].sort_values('ds')
        y_train = train_item['y'].values
        
        # Calculate metrics
        mase = calculate_mase(y_true, y_pred, y_train, seasonality=1)
        rmsse = calculate_rmsse(y_true, y_pred, y_train, seasonality=1)
        
        if not np.isnan(mase):
            mase_scores.append(mase)
        if not np.isnan(rmsse):
            rmsse_scores.append(rmsse)
    
    return {
        'mase': np.mean(mase_scores),
        'rmsse': np.mean(rmsse_scores)
    }

print("✅ Evaluation function defined")
print("\nThis function calculates:")
print("  - MASE: Mean Absolute Scaled Error")
print("  - RMSSE: Root Mean Squared Scaled Error")
print("\nBoth metrics are averaged across all items")

## 9. Evaluate Simple and Statistical Models

Let's calculate MASE and RMSSE for all models trained so far.

In [ ]:
# Evaluate simple models
print("Evaluating simple baseline models...\n")

results_simple = {}
model_names_simple = ['Naive', 'SeasonalNaive', 'WindowAverage', 'SeasWA']

for model_name in model_names_simple:
    metrics = evaluate_model_cv(forecasts_simple, df_test_sf, df_train_sf, model_name)
    results_simple[model_name] = metrics
    print(f"{model_name:20s} - MASE: {metrics['mase']:.4f}, RMSSE: {metrics['rmsse']:.4f}")

# Evaluate AutoETS
print("\nEvaluating AutoETS...\n")
metrics_ets = evaluate_model_cv(forecasts_ets, df_test_sf, df_train_sf, 'AutoETS')
results_simple['AutoETS'] = metrics_ets
print(f"AutoETS              - MASE: {metrics_ets['mase']:.4f}, RMSSE: {metrics_ets['rmsse']:.4f}")

print("\n✅ All statistical models evaluated!")

In [ ]:
# Visualize results
results_df = pd.DataFrame(results_simple).T
results_df = results_df.sort_values('mase')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MASE
axes[0].barh(results_df.index, results_df['mase'], color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(x=1.0, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Naive Baseline')
axes[0].set_xlabel('MASE (Lower is Better)', fontsize=11)
axes[0].set_title('Mean Absolute Scaled Error', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3, axis='x')

# RMSSE
axes[1].barh(results_df.index, results_df['rmsse'], color='coral', alpha=0.7, edgecolor='black')
axes[1].axvline(x=1.0, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Naive Baseline')
axes[1].set_xlabel('RMSSE (Lower is Better)', fontsize=11)
axes[1].set_title('Root Mean Squared Scaled Error', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3, axis='x')

plt.suptitle('Simple and Statistical Models Performance', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Key Observations:")
print("- Models with MASE/RMSSE < 1.0 outperform naive forecast")
print("- Seasonal models typically perform better due to weekly patterns")
print("- AutoETS automatically adapts to each series' characteristics")

## 10. Global ML Models - Lag Selection

Now let's explore **global machine learning models** using **MLForecast**.

First, we need to find the optimal number of lags through cross-validation.

In [ ]:
# Prepare data for MLForecast
df_train_mlf = df_train_sf.copy()  # Already renamed to 'ds'
df_test_mlf = df_test_sf.copy()

# Test different lag configurations
max_lags_candidates = [7, 14, 21, 28]

print("Testing different lag configurations via cross-validation...\n")
print("This may take a few minutes...\n")

cv_results_lr = {}
cv_results_lgbm = {}

for max_lags in max_lags_candidates:
    print(f"Testing max_lags={max_lags}...")
    
    lags = list(range(1, max_lags + 1))
    
    # Linear Regression
    fcst_lr = MLForecast(
        models={'LinearRegression': LinearRegression()},
        freq='D',
        lags=lags
    )
    
    cv_lr = fcst_lr.cross_validation(
        df=df_train_mlf,
        h=7,
        n_windows=2
    )
    
    mse_lr = mean_squared_error(cv_lr['y'], cv_lr['LinearRegression'])
    cv_results_lr[max_lags] = mse_lr
    
    # LightGBM
    fcst_lgbm = MLForecast(
        models={'LightGBM': lgb.LGBMRegressor(verbose=-1)},
        freq='D',
        lags=lags
    )
    
    cv_lgbm = fcst_lgbm.cross_validation(
        df=df_train_mlf,
        h=7,
        n_windows=2
    )
    
    mse_lgbm = mean_squared_error(cv_lgbm['y'], cv_lgbm['LightGBM'])
    cv_results_lgbm[max_lags] = mse_lgbm

best_lags_lr = min(cv_results_lr, key=cv_results_lr.get)
best_lags_lgbm = min(cv_results_lgbm, key=cv_results_lgbm.get)

print("\n✅ Lag selection complete!")
print(f"\nOptimal lags:")
print(f"  Linear Regression: {best_lags_lr} (MSE: {cv_results_lr[best_lags_lr]:.2f})")
print(f"  LightGBM: {best_lags_lgbm} (MSE: {cv_results_lgbm[best_lags_lgbm]:.2f})")

In [ ]:
# Visualize lag selection results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

lags_list = sorted(cv_results_lr.keys())

# Linear Regression
mse_list_lr = [cv_results_lr[lag] for lag in lags_list]
axes[0].plot(lags_list, mse_list_lr, marker='o', linewidth=2, markersize=8, color='#1f77b4')
axes[0].axvline(x=best_lags_lr, color='red', linestyle='--', linewidth=2, alpha=0.7,
                label=f'Optimal: {best_lags_lr}')
axes[0].set_xlabel('max_lags', fontsize=11)
axes[0].set_ylabel('Validation MSE', fontsize=11)
axes[0].set_title('Linear Regression: Lag Selection', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# LightGBM
mse_list_lgbm = [cv_results_lgbm[lag] for lag in lags_list]
axes[1].plot(lags_list, mse_list_lgbm, marker='o', linewidth=2, markersize=8, color='#2ca02c')
axes[1].axvline(x=best_lags_lgbm, color='red', linestyle='--', linewidth=2, alpha=0.7,
                label=f'Optimal: {best_lags_lgbm}')
axes[1].set_xlabel('max_lags', fontsize=11)
axes[1].set_ylabel('Validation MSE', fontsize=11)
axes[1].set_title('LightGBM: Lag Selection', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Optimal Lag Selection via Cross-Validation', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("- Too few lags miss important patterns")
print("- Too many lags add noise and computational cost")
print("- Cross-validation helps find the sweet spot!")

## 11. Global ML Models - Feature Engineering

Now let's add feature engineering:
1. **Date Features**: day_of_week, day_of_month, week_of_year, is_weekend
2. **Normalization**: StandardScaler and LocalStandardScaler

We'll test different combinations for LightGBM.

In [ ]:
# Define custom date feature
def is_weekend(dates):
    return (dates.dayofweek >= 5).astype(int)

# Test different feature engineering configurations
fe_configs = {
    'Baseline (no FE)': {
        'target_transforms': None,
        'date_features': None
    },
    'With Scaling': {
        'target_transforms': [LocalStandardScaler()],
        'date_features': None
    },
    'With Date Features': {
        'target_transforms': None,
        'date_features': ['dayofweek', 'day', 'week', is_weekend]
    },
    'With Both': {
        'target_transforms': [LocalStandardScaler()],
        'date_features': ['dayofweek', 'day', 'week', is_weekend]
    }
}

print("Testing feature engineering configurations for LightGBM...\n")

fe_results = {}
fe_forecasts = {}

for config_name, config in fe_configs.items():
    print(f"  {config_name}...")
    
    # Initialize MLForecast
    fcst = MLForecast(
        models={'LightGBM': lgb.LGBMRegressor(n_estimators=100, max_depth=10, random_state=42, verbose=-1)},
        freq='D',
        lags=list(range(1, best_lags_lgbm + 1)),
        target_transforms=config['target_transforms'],
        date_features=config['date_features']
    )
    
    # Fit and predict
    fcst.fit(df_train_mlf)
    forecasts = fcst.predict(test_size)
    fe_forecasts[config_name] = forecasts
    
    # Evaluate
    metrics = evaluate_model_cv(forecasts, df_test_mlf, df_train_mlf, 'LightGBM')
    fe_results[config_name] = metrics

print("\n✅ Feature engineering comparison complete!")

# Print results
print("\nFeature Engineering Impact: LightGBM")
print("="*60)
print(f"{'Configuration':<25} {'MASE':<12} {'RMSSE':<12}")
print("-"*60)
for config_name, metrics in fe_results.items():
    print(f"{config_name:<25} {metrics['mase']:<12.4f} {metrics['rmsse']:<12.4f}")

# Calculate improvement
baseline_mase = fe_results['Baseline (no FE)']['mase']
best_mase = fe_results['With Both']['mase']
improvement = ((baseline_mase - best_mase) / baseline_mase) * 100

print(f"\n💡 Improvement: {improvement:.1f}% reduction in MASE with full feature engineering!")

In [ ]:
# Visualize feature engineering impact
fe_results_df = pd.DataFrame(fe_results).T

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

config_names = list(fe_configs.keys())
colors = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']

# MASE
axes[0].bar(range(len(config_names)), fe_results_df['mase'], 
            color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_xticks(range(len(config_names)))
axes[0].set_xticklabels(config_names, rotation=15, ha='right')
axes[0].set_ylabel('MASE (Lower is Better)', fontsize=11)
axes[0].set_title('Impact of Feature Engineering on MASE', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, val in enumerate(fe_results_df['mase']):
    axes[0].text(i, val, f'{val:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# RMSSE
axes[1].bar(range(len(config_names)), fe_results_df['rmsse'], 
            color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1].set_xticks(range(len(config_names)))
axes[1].set_xticklabels(config_names, rotation=15, ha='right')
axes[1].set_ylabel('RMSSE (Lower is Better)', fontsize=11)
axes[1].set_title('Impact of Feature Engineering on RMSSE', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, val in enumerate(fe_results_df['rmsse']):
    axes[1].text(i, val, f'{val:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.suptitle('Feature Engineering Impact on LightGBM Performance', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n✅ Feature engineering significantly improves performance!")
print("   → Scaling helps standardize features")
print("   → Date features capture calendar effects (weekends, etc.)")
print("   → Combining both gives best results!")

## 12. Train Final Models with Best Configuration

Now let's train Linear Regression and LightGBM with their optimal configurations.

In [ ]:
print("Training final models with optimal configurations...\n")

# Linear Regression (with best lags + feature engineering)
print(f"Training Linear Regression (max_lags={best_lags_lr})...")
fcst_lr_final = MLForecast(
    models={'LinearRegression': LinearRegression()},
    freq='D',
    lags=list(range(1, best_lags_lr + 1)),
    target_transforms=[LocalStandardScaler()],
    date_features=['dayofweek', 'day', 'week', is_weekend]
)
fcst_lr_final.fit(df_train_mlf)
forecasts_lr_final = fcst_lr_final.predict(test_size)

# LightGBM (with best lags + feature engineering)
print(f"Training LightGBM (max_lags={best_lags_lgbm})...")
fcst_lgbm_final = MLForecast(
    models={'LightGBM': lgb.LGBMRegressor(n_estimators=100, max_depth=10, random_state=42, verbose=-1)},
    freq='D',
    lags=list(range(1, best_lags_lgbm + 1)),
    target_transforms=[LocalStandardScaler()],
    date_features=['dayofweek', 'day', 'week', is_weekend]
)
fcst_lgbm_final.fit(df_train_mlf)
forecasts_lgbm_final = fcst_lgbm_final.predict(test_size)

print("\n✅ Final models trained!")

## 13. Final Model Comparison

Let's compare all models using both MASE and RMSSE metrics.

In [ ]:
# Evaluate final ML models
metrics_lr_final = evaluate_model_cv(forecasts_lr_final, df_test_mlf, df_train_mlf, 'LinearRegression')
metrics_lgbm_final = evaluate_model_cv(forecasts_lgbm_final, df_test_mlf, df_train_mlf, 'LightGBM')

# Combine all results
all_results = results_simple.copy()
all_results['LinearRegression'] = metrics_lr_final
all_results['LightGBM'] = metrics_lgbm_final

# Create comparison DataFrame
comparison_df = pd.DataFrame(all_results).T
comparison_df = comparison_df.sort_values('mase')

print("\n" + "="*80)
print("FINAL MODEL COMPARISON: ALL MODELS")
print("="*80)
print(f"{'Model':<25} {'MASE':<15} {'RMSSE':<15} {'Beats Naive?':<15}")
print("-"*80)

for model, metrics in comparison_df.iterrows():
    beats_naive = '✅ Yes' if metrics['mase'] < 1.0 else '❌ No'
    print(f"{model:<25} {metrics['mase']:<15.4f} {metrics['rmsse']:<15.4f} {beats_naive:<15}")

print("="*80)

# Identify best model
best_model_mase = comparison_df['mase'].idxmin()
best_model_rmsse = comparison_df['rmsse'].idxmin()

print(f"\n🏆 Best Model by MASE: {best_model_mase} ({comparison_df.loc[best_model_mase, 'mase']:.4f})")
print(f"🏆 Best Model by RMSSE: {best_model_rmsse} ({comparison_df.loc[best_model_rmsse, 'rmsse']:.4f})")

In [ ]:
# Visualize final comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# MASE comparison
colors_mase = ['green' if x < 1.0 else 'red' for x in comparison_df['mase']]
axes[0].barh(comparison_df.index, comparison_df['mase'], 
             color=colors_mase, alpha=0.7, edgecolor='black', linewidth=1.2)
axes[0].axvline(x=1.0, color='black', linestyle='--', linewidth=2, 
                alpha=0.5, label='Naive Baseline (1.0)')
axes[0].set_xlabel('MASE (Lower is Better)', fontsize=11)
axes[0].set_title('Mean Absolute Scaled Error', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3, axis='x')

# RMSSE comparison
colors_rmsse = ['green' if x < 1.0 else 'red' for x in comparison_df['rmsse']]
axes[1].barh(comparison_df.index, comparison_df['rmsse'], 
             color=colors_rmsse, alpha=0.7, edgecolor='black', linewidth=1.2)
axes[1].axvline(x=1.0, color='black', linestyle='--', linewidth=2, 
                alpha=0.5, label='Naive Baseline (1.0)')
axes[1].set_xlabel('RMSSE (Lower is Better)', fontsize=11)
axes[1].set_title('Root Mean Squared Scaled Error', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3, axis='x')

plt.suptitle('Final Model Comparison: All Models', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("  - Green bars: Models that beat naive baseline (metric < 1.0)")
print("  - Red bars: Models worse than naive baseline")
print("  - Lower values are better for both metrics")
print("  - MASE focuses on absolute errors, RMSSE penalizes large errors more")

## 14. Forecast Visualizations

### 14.1 Individual Item Forecasts

Let's visualize forecasts for a sample of items using the best model.

In [ ]:
# Select best model forecasts
if best_model_mase == 'LightGBM':
    best_forecasts = forecasts_lgbm_final
    best_model_col = 'LightGBM'
elif best_model_mase == 'LinearRegression':
    best_forecasts = forecasts_lr_final
    best_model_col = 'LinearRegression'
else:
    # Use AutoETS if it's the best
    best_forecasts = forecasts_ets
    best_model_col = 'AutoETS'

# Visualize 8 sample items
sample_items_viz = items[::len(items)//8][:8]

fig, axes = plt.subplots(4, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, item in enumerate(sample_items_viz):
    # Get data for this item
    train_item = df_train_mlf[df_train_mlf['unique_id'] == item].sort_values('ds')
    test_item = df_test_mlf[df_test_mlf['unique_id'] == item].sort_values('ds')
    
    if 'ds' in best_forecasts.columns:
        forecast_item = best_forecasts.reset_index()
        forecast_item = forecast_item[forecast_item['unique_id'] == item].sort_values('ds')
    else:
        forecast_item = best_forecasts[best_forecasts['unique_id'] == item].sort_values('ds')
    
    # Plot last 90 days of training
    train_context = train_item.tail(90)
    axes[idx].plot(train_context['ds'], train_context['y'], 
                   color='black', linewidth=1, alpha=0.7, label='Train')
    
    # Plot actual test data
    axes[idx].plot(test_item['ds'], test_item['y'], 
                   color='black', linewidth=1.5, alpha=0.8, 
                   marker='o', markersize=3, label='Actual')
    
    # Plot forecast
    axes[idx].plot(forecast_item['ds'], forecast_item[best_model_col], 
                   color='red', linewidth=2, linestyle='--', alpha=0.8, label='Forecast')
    
    # Mark split
    axes[idx].axvline(x=split_date, color='gray', linestyle='-', linewidth=1, alpha=0.5)
    
    axes[idx].set_title(f'{item}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=8)
    axes[idx].set_ylabel('Sales', fontsize=8)
    axes[idx].tick_params(labelsize=7)
    axes[idx].legend(fontsize=7, loc='upper left')
    axes[idx].grid(True, alpha=0.3)

plt.suptitle(f'Individual Item Forecasts: {best_model_mase}', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

### 14.2 Aggregated Forecasts

Let's aggregate forecasts across all items and compare to aggregated actuals.

In [ ]:
# Aggregate forecasts and actuals
if 'ds' in best_forecasts.columns:
    forecasts_agg = best_forecasts.reset_index().groupby('ds')[best_model_col].sum().reset_index()
else:
    forecasts_agg = best_forecasts.groupby('ds')[best_model_col].sum().reset_index()

actuals_agg = df_test_mlf.groupby('ds')['y'].sum().reset_index()
train_agg = df_train_mlf.groupby('ds')['y'].sum().reset_index()

# Plot aggregated forecast
fig, ax = plt.subplots(figsize=(14, 6))

# Plot last 90 days of training
train_context = train_agg.tail(90)
ax.plot(train_context['ds'], train_context['y'], 
        color='black', linewidth=1, alpha=0.7, label='Train (Aggregated)')

# Plot actual test data (aggregated)
ax.plot(actuals_agg['ds'], actuals_agg['y'], 
        color='black', linewidth=2, alpha=0.8, marker='o', markersize=5, label='Actual (Aggregated)')

# Plot forecast (aggregated)
ax.plot(forecasts_agg['ds'], forecasts_agg[best_model_col], 
        color='red', linewidth=2.5, linestyle='--', alpha=0.8, label=f'Forecast (Aggregated)')

# Mark split
ax.axvline(x=split_date, color='gray', linestyle='-', linewidth=1.5, alpha=0.5, label='Train/Test Split')

ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Total Daily Sales', fontsize=11)
ax.set_title(f'Aggregated Forecasts Across All Items: {best_model_mase}', fontsize=13, fontweight='bold')
ax.legend(fontsize=10, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate aggregated error
agg_mae = mean_absolute_error(actuals_agg['y'], forecasts_agg[best_model_col])
agg_mape = np.mean(np.abs((actuals_agg['y'] - forecasts_agg[best_model_col]) / actuals_agg['y'])) * 100

print(f"\nAggregated Forecast Performance:")
print(f"  MAE: {agg_mae:.2f}")
print(f"  MAPE: {agg_mape:.2f}%")

### 14.3 Error Distribution Analysis

In [ ]:
# Calculate errors for all items
if 'ds' in best_forecasts.columns:
    errors_df = best_forecasts.reset_index().merge(
        df_test_mlf[['unique_id', 'ds', 'y']], 
        on=['unique_id', 'ds']
    )
else:
    errors_df = best_forecasts.merge(
        df_test_mlf[['unique_id', 'ds', 'y']], 
        on=['unique_id', 'ds']
    )

errors_df['error'] = errors_df['y'] - errors_df[best_model_col]
errors_df['abs_error'] = np.abs(errors_df['error'])
errors_df['pct_error'] = (errors_df['error'] / errors_df['y']) * 100

# Visualize error distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Error histogram
axes[0, 0].hist(errors_df['error'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[0, 0].set_xlabel('Forecast Error', fontsize=10)
axes[0, 0].set_ylabel('Frequency', fontsize=10)
axes[0, 0].set_title('Error Distribution', fontsize=11, fontweight='bold')
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# Absolute error by item
item_errors = errors_df.groupby('unique_id')['abs_error'].mean().sort_values(ascending=False).head(20)
axes[0, 1].barh(range(len(item_errors)), item_errors.values, color='coral', alpha=0.7, edgecolor='black')
axes[0, 1].set_yticks(range(len(item_errors)))
axes[0, 1].set_yticklabels(item_errors.index, fontsize=7)
axes[0, 1].set_xlabel('Mean Absolute Error', fontsize=10)
axes[0, 1].set_title('Top 20 Items by Absolute Error', fontsize=11, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Actual vs Predicted scatter
axes[1, 0].scatter(errors_df['y'], errors_df[best_model_col], 
                   alpha=0.3, s=10, color='steelblue')
max_val = max(errors_df['y'].max(), errors_df[best_model_col].max())
axes[1, 0].plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Forecast')
axes[1, 0].set_xlabel('Actual', fontsize=10)
axes[1, 0].set_ylabel('Predicted', fontsize=10)
axes[1, 0].set_title('Actual vs Predicted', fontsize=11, fontweight='bold')
axes[1, 0].legend(fontsize=9)
axes[1, 0].grid(True, alpha=0.3)

# Error over time
error_by_date = errors_df.groupby('ds')['error'].mean()
axes[1, 1].plot(error_by_date.index, error_by_date.values, 
                color='steelblue', linewidth=2, marker='o', markersize=5)
axes[1, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Date', fontsize=10)
axes[1, 1].set_ylabel('Mean Error', fontsize=10)
axes[1, 1].set_title('Forecast Error Over Time', fontsize=11, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
plt.xticks(rotation=45)

plt.suptitle('Error Analysis', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print(f"\n📊 Error Statistics:")
print(f"  Mean Error: {errors_df['error'].mean():.2f}")
print(f"  Std Error: {errors_df['error'].std():.2f}")
print(f"  Mean Absolute Error: {errors_df['abs_error'].mean():.2f}")
print(f"  Median Absolute Error: {errors_df['abs_error'].median():.2f}")

## 15. Summary and Recommendations

### What We Accomplished:

1. ✅ **Exploratory Data Analysis**
   - Visualized time series patterns
   - Decomposed into trend, seasonality, and residuals
   - Measured seasonality strength (average: {:.3f})
   - Identified weekly patterns with peak sales on weekends

2. ✅ **Simple Baseline Models**
   - Established benchmarks with Naive, Seasonal Naive, Window Average
   - Seasonal models performed better due to strong weekly patterns

3. ✅ **Statistical Models**
   - AutoETS automatically selected optimal components
   - Captured both trend and seasonality

4. ✅ **Custom Metrics**
   - Implemented MASE (Mean Absolute Scaled Error)
   - Implemented RMSSE (Root Mean Squared Scaled Error)
   - Both metrics scale-free and compare against naive baseline

5. ✅ **Global ML Models**
   - Performed lag selection via cross-validation
   - Applied feature engineering (date features + normalization)
   - Trained Linear Regression and LightGBM

6. ✅ **Model Comparison**
   - Evaluated all models using MASE and RMSSE
   - Best model: **{best_model_mase}** (MASE: {comparison_df.loc[best_model_mase, 'mase']:.4f})
   - Visualized forecasts and error distributions

### Key Insights:

- **Seasonality Matters**: Models that capture weekly patterns perform significantly better
- **Feature Engineering**: Date features and normalization improve performance by ~{improvement:.1f}%
- **Lag Selection**: Cross-validation helps find optimal historical window
- **Global Models**: Train once on all series, more efficient than local models
- **Scaled Metrics**: MASE and RMSSE provide interpretable, scale-free evaluation

### Recommendations:

1. **For Production**: Use **{best_model_mase}** with:
   - Optimal lags: {best_lags_lgbm if best_model_mase == 'LightGBM' else best_lags_lr}
   - LocalStandardScaler for normalization
   - Date features: day_of_week, day, week, is_weekend

2. **For Experimentation**:
   - Try more sophisticated models (ARIMA, Prophet)
   - Add exogenous features (holidays, promotions, weather)
   - Experiment with different loss functions

3. **For Monitoring**:
   - Track MASE and RMSSE over time
   - Retrain when performance degrades
   - Monitor for distribution shifts

### Next Steps:

- 🔄 Implement automated retraining pipeline
- 📊 Add prediction intervals for uncertainty quantification
- 🚀 Deploy model to production environment
- 📈 Set up monitoring dashboards

---

### 🎓 Congratulations!

You've completed a full end-to-end time series forecasting workflow! 🚀

In [ ]:
# Print final summary with actual values
print("="*80)
print("                    WORKSHOP SUMMARY                    ")
print("="*80)
print(f"\n📊 Dataset: Top 50 Food Items")
print(f"   - Items: {df['unique_id'].nunique()}")
print(f"   - Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   - Total observations: {len(df):,}")

print(f"\n🎯 Best Model: {best_model_mase}")
print(f"   - MASE: {comparison_df.loc[best_model_mase, 'mase']:.4f}")
print(f"   - RMSSE: {comparison_df.loc[best_model_mase, 'rmsse']:.4f}")
print(f"   - Performance: {'✅ Beats Naive' if comparison_df.loc[best_model_mase, 'mase'] < 1.0 else '❌ Below Naive'}")

print(f"\n📈 Average Seasonality Strength: {strength_df['Seasonality_Strength'].mean():.3f}")
print(f"   - Strong weekly patterns detected")
print(f"   - Peak sales day: {peak_day}")

print(f"\n🔧 Optimal Configuration:")
if best_model_mase == 'LightGBM':
    print(f"   - Model: LightGBM")
    print(f"   - Lags: {best_lags_lgbm}")
    print(f"   - Feature Engineering: LocalStandardScaler + Date Features")
elif best_model_mase == 'LinearRegression':
    print(f"   - Model: Linear Regression")
    print(f"   - Lags: {best_lags_lr}")
    print(f"   - Feature Engineering: LocalStandardScaler + Date Features")
else:
    print(f"   - Model: {best_model_mase}")
    print(f"   - Automatic component selection")

print(f"\n💡 Key Takeaways:")
print(f"   1. Feature engineering improves performance by ~{improvement:.1f}%")
print(f"   2. Cross-validation essential for hyperparameter tuning")
print(f"   3. Scaled metrics (MASE, RMSSE) provide interpretable evaluation")
print(f"   4. Global models more efficient than local approaches")

print("\n" + "="*80)
print("               ✅ WORKSHOP COMPLETE! 🎉")
print("="*80)